In [4]:
import ee
import geemap
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import os

# Authenticate and Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define the study area (Lankien, South Sudan)
lankien_bbox = ee.Geometry.Rectangle([30.957, 7.480, 32.008, 9.205])

# Load MODIS MOD13Q1 EVI Data (16-day composite, 250m resolution)
dataset = ee.ImageCollection('MODIS/061/MOD13Q1') \
    .filterBounds(lankien_bbox) \
    .filterDate('2015-11-01', '2025-01-01') \
    .select(['EVI'])

# Function to scale EVI values (MODIS stores them multiplied by 10,000)
def scale_evi(image):
    return image.multiply(0.0001).copyProperties(image, ['system:time_start'])

# Apply scaling function
dataset = dataset.map(scale_evi)

# Function to extract EVI value from Lankien
def extract_evi(image):
    evi_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=lankien_bbox,
        scale=250,
        bestEffort=True,
        maxPixels=1e13
    ).get('EVI')
    
    return ee.Feature(None, {
        'EVI': evi_value,
        'date': image.date().format('YYYY-MM-dd')
    })

# Convert ImageCollection to FeatureCollection for export
evi_features = dataset.map(extract_evi)
evi_list = evi_features.aggregate_array("EVI").getInfo()
dates = evi_features.aggregate_array("date").getInfo()

# Convert to Pandas DataFrame (16-day EVI)
df_16day = pd.DataFrame({'date': dates, 'EVI': evi_list})
df_16day['date'] = pd.to_datetime(df_16day['date'])
df_16day = df_16day.dropna()  # Remove missing values

# Generate daily date range
full_dates = pd.date_range(start=df_16day['date'].min(), end=df_16day['date'].max(), freq='D')

# Perform Interpolation (Linear or Spline)
interp_func = interp1d(df_16day['date'].astype(int), df_16day['EVI'], kind='linear', fill_value="extrapolate")
df_daily = pd.DataFrame({'date': full_dates})
df_daily['EVI'] = interp_func(df_daily['date'].astype(int))

# Save both DataFrames as CSV files in "EVI/" folder
output_folder = "EVI"
os.makedirs(output_folder, exist_ok=True)

df_16day.to_csv(os.path.join(output_folder, "EVI_16day_Lankien.csv"), index=False)
df_daily.to_csv(os.path.join(output_folder, "EVI_daily_Lankien.csv"), index=False)

# Print first few rows
print("16-day MODIS EVI Data (Original):")
print(df_16day.head())

print("\nDaily EVI Data (Interpolated):")
print(df_daily.head())


16-day MODIS EVI Data (Original):
        date       EVI
0 2015-11-01  0.351557
1 2015-11-17  0.259255
2 2015-12-03  0.232601
3 2015-12-19  0.167840
4 2016-01-01  0.160166

Daily EVI Data (Interpolated):
        date       EVI
0 2015-11-01  0.351557
1 2015-11-02  0.345788
2 2015-11-03  0.340020
3 2015-11-04  0.334251
4 2015-11-05  0.328482
